# Compute pRF cortical magnification base on pRF parameters 

Based on previous work (see : https://github.com/mszinte/pRF_analysis/blob/main/analysis_code/postproc/prf/postfit/compute_css_pcm.py) we want to apply a similar way to compute pRF CM on the HCP data_set 

TO DO :
- [ ] acces HCP data
- [ ] understand strucuture of HCP data and neuropythy
- [ ] compute CM for vertex in V1 